<div style="background-color:#2b0000; color:white; padding:25px; border-radius:10px; 
            text-align:center; font-family:'Segoe UI', sans-serif;">

  <h1 style="margin-bottom:8px;"> Cars 4 You 🏎️💨</h1>
  <h3 style="margin-top:0; font-style:italic; font-weight:normal; color:#f05a5a;">
    Auxiliary Notebook – Nominal Variables Fixing
  </h3>

  <hr style="width:60%; border:1px solid #700000; margin:15px auto;">

  <p style="margin:5px 0; font-size:15px;">
    <b>Group 4</b> - Machine Learning Project (2025/2026)
  </p>
  <p style="margin:0; font-size:13px; color:#e3bdbd;">
    Master in Data Science and Advanced Analytics - Nova Information Management School
  </p>
</div>

<br>

<div style="background-color:#3a0808; color:#f4eaea; padding:15px 20px; border-left:5px solid #700000; 
            border-radius:6px; font-family:'Segoe UI', sans-serif; font-size:14px;">

  <b>Notebook Context</b><br>
  Due to the large extent of this subchapter, this encoding process is handled in a separate notebook.  
  In the end, the necessary dictionaries will be exported to ensure consistent label conversion 
  across all notebooks.

  <br>
  There are four nominal variables in the dataset that require preprocessing prior to outlier detection, 
  as under-represented anomalies could otherwise distort the results:
  <ul style="margin-top:5px; margin-bottom:0;">
    <li><code>Brand</code> – The brand of the car.</li>
    <li><code>Model</code> – The specific car model.</li>
    <li><code>fuelType</code> – The type of fuel the car uses.</li>
    <li><code>transmission</code> – The type of transmission system.</li>
  </ul>
</div>

<br>

<div style="text-align:center; margin-top:10px;">
  <a href="main.ipynb" 
     style="display:inline-block; background-color:#700000; color:#fff; 
            padding:8px 16px; text-decoration:none; border-radius:6px; 
            font-family:'Segoe UI', sans-serif; font-size:13px;">
     <- Back to Main Notebook
  </a>
</div>

<br>

<div style="text-align:right; font-size:12px; color:#d8bfbf;">
  Last updated: November 2025
</div>


# 1. Import the needed libraries

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

It was stated that we could merge both datasets to facilitate this task. There are misspelling variations present in one dataset that are not in the other. By merging both datasets, we can ensure that all unique values are considered when creating the mapping.

In [2]:
df1 = pd.read_csv('data/train.csv')
df2 = pd.read_csv('data/test.csv')
df = pd.concat([df1, df2], ignore_index=True)

## 2.1. Fix Categorical Variables

Due to the large extent of this subchapter, we decided to do this procedure in a separate notebook. In the end we will export the necessary dictionaries to convert the labels properly.

There are 4 different nominal variables in the dataset:

- `Brand`: The brand of the car.
  
- `Model`: The model of the car.

- `fuelType`: The type of fuel the car requires.

- `transmission`: The type of trasmission the car has.

This needs to be handled prior to outliers detection, since there are currently anomalies which would jeopardize the data integrity.


### 2.1.1. Preparations

In this step, we define a **reusable pipeline** and a set of **helper functions** to clean categorical variables. This dataset contains inconsistent entries for the same category (e.g., *"Mercedez"*, *"mercedes-benz"*, *"MERCEDES"*). 



In [3]:
def vc_column(df, column):
    return df[column].value_counts()

To standardize these, we use **Cosine Similarity** on **character-level TF–IDF embeddings**, which helps identify the closest canonical value for each noisy or misspelled variation.

- **TF–IDF (Term Frequency–Inverse Document Frequency)** transforms text into numerical vectors that reflect how relevant certain character patterns (n-grams) are within each string.
- **Cosine Similarity** then measures the *angular distance* between these vectors — two strings that are more similar will have vectors that point in nearly the same direction (i.e., a cosine similarity close to **1**).
- By comparing each variation to a list of canonical values, we can automatically assign each noisy value to the best matching standardized form.

This approach is especially effective for cleaning text-based categorical data where typos, capitalization, or minor spelling differences are common. We also found that **mapping to the closest neighbour (instead of blindly clustering), although more time-consuming, proved to generate a cleaner dataset**. 

We use a merge of both train and test to get possibly miss-spellings and fix right away.

In [4]:
def build_sim_mapping(df:pd.DataFrame, 
                      column_name:str, 
                      canonical_values:list, 
                      ngram_range=(2, 5)) -> pd.DataFrame:
    """
    Builds a mapping between noisy/variant categorical values and their canonical equivalents
    using cosine similarity over TF-IDF character n-grams.
    """

    # Get unique non-null variations
    all_variations = df[column_name].dropna().unique()

    # Vectorize all text values using TF-IDF of character n-grams
    vectorizer = TfidfVectorizer(analyzer="char", ngram_range=ngram_range)
    X_all = vectorizer.fit_transform(all_variations)
    X_canon = vectorizer.transform(canonical_values)

    # Compute cosine similarity between each variation and canonical form
    similarity_matrix = cosine_similarity(X_all, X_canon)

    # Assign each variation to the closest canonical value
    closest_idx = similarity_matrix.argmax(axis=1)
    assigned_values = [canonical_values[i] for i in closest_idx]

    # Create mapping dataframe
    mapping_df = pd.DataFrame({
        "Variation": all_variations,
        "AssignedValue": assigned_values
    })

    return mapping_df

### 2.1.2. `Brand` Feature

After careful analysis, we realized that the dataset only contains 9 canonical brands. The rest is miss-spelled.

In [5]:
diff = df['Brand'].nunique() - df1['Brand'].nunique()
print(f"Number of unique Brand values in total concatenated that are not in train: {diff}") # proving that merging is necessary

Number of unique Brand values in total concatenated that are not in train: 4


In [6]:
vc_column(df, 'Brand').head(15)

Brand
Ford        21168
Mercedes    15372
VW          13983
Opel        12318
BMW          9945
Audi         9646
Toyota       6093
Skoda        5664
Hyundai      4391
FORD          450
ord           446
For           433
ford          423
MERCEDES      322
mercedes      321
Name: count, dtype: int64

In [7]:
canonical_brands = ["Ford", "Mercedes", "VW", "Opel", "BMW", "Audi", "Toyota", "Skoda", "Hyundai"]

brand_mapping = build_sim_mapping(df, 'Brand', canonical_brands)

#### Manually inspect and correct any misassignments below

In [8]:
brand_mapping.tail(20)

,Variation,AssignedValue
56,OYOTA,Toyota
57,HYUNDA,Hyundai
58,mw,BMW
59,OPE,Opel
60,mercede,Mercedes
61,ERCEDES,Mercedes
62,ercede,Mercedes
63,TOYOT,Toyota
64,MERCEDE,Mercedes
65,ORD,Ford


Appears that Ford has assignments which are not correct. For example, "M" (line 72) appears to be wrong.

In [9]:
brand_mapping[brand_mapping["AssignedValue"] == "Ford"]

,Variation,AssignedValue
3,Ford,Ford
8,FOR,Ford
11,w,Ford
12,ord,Ford
21,V,Ford
24,For,Ford
31,FORD,Ford
33,W,Ford
38,ford,Ford
47,v,Ford


In [10]:
df[df["Brand"].isin(["w", "V", "W", "v"])]

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
24,68197,w,Golf,2013.0,12299.0,Automatic,64739.0,Petrol,160.0,44.1,2.0,47.0,4.0,0.0
239,71915,V,Golf,2018.0,15561.0,Manual,34535.0,Petrol,145.0,55.4,1.5,35.0,0.0,0.0
413,71027,V,Polo,2016.0,9498.0,Manual,7157.0,Petrol,20.0,60.1,1.0,30.0,4.0,0.0
686,70759,V,Golf,2017.0,17950.0,manual,26000.0,Diesel,145.0,60.1,2.0,84.0,1.0,0.0
767,72396,W,Tiguan,2015.0,16291.0,Semi-Auto,46647.0,Diesel,160.0,49.6,2.0,56.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107868,105957,W,Arteon,2018.0,NaN,Semi-Auto,5952.0,Petrol,145.0,48.7,1.5,53.0,2.0,0.0
107977,104285,V,NaN,2014.0,NaN,Manual,36934.0,Petrol,20.0,62.8,1.0,44.0,0.0,0.0
108120,104340,W,Golf,2017.0,NaN,Manual,15712.0,Diesel,0.0,74.3,1.6,42.0,1.0,0.0
108281,105293,W,Passat,2016.0,NaN,Manual,109000.0,Diesel,20.0,67.3,2.0,76.0,3.0,0.0


In [11]:
brand_mapping.loc[brand_mapping["Variation"] == "w", "AssignedValue"] = "VW"
brand_mapping.loc[brand_mapping["Variation"] == "V", "AssignedValue"] = "VW"
brand_mapping.loc[brand_mapping["Variation"] == "W", "AssignedValue"] = "VW"
brand_mapping.loc[brand_mapping["Variation"] == "v", "AssignedValue"] = "VW"

In [12]:
df[df["Brand"]=="M"]

,carID,Brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners,hasDamage
86582,79855,M,1 Series,2018.0,NaN,Semi-Auto,24275.0,Diesel,145.0,68.9,1.5,94.0,3.0,0.0
99481,79848,M,2 Series,2020.0,NaN,semi-auto,5.0,Diesel,145.0,67.3,2.0,NaN,2.0,0.0


In [13]:
brand_mapping.loc[brand_mapping["Variation"] == "M", "AssignedValue"] = "BMW"

Verifying the changes:

In [14]:
brand_mapping[brand_mapping["AssignedValue"] == "VW"]

,Variation,AssignedValue
0,VW,VW
11,w,VW
21,V,VW
33,W,VW
35,vw,VW
47,v,VW


In [15]:
brand_mapping[brand_mapping["AssignedValue"] == "BMW"]

,Variation,AssignedValue
4,BMW,BMW
13,MW,BMW
14,bmw,BMW
16,BM,BMW
58,mw,BMW
72,M,BMW
75,bm,BMW


Now (hypothetically) we can replace on the dataframes. In order to later replace correctly, we will save this dataframe as csv.

In [16]:
brand_mapping.to_csv('./mapping_dicts/brand_mapping.csv', index=False)

### 2.1.3. `fuelType` Feature

After assessing the values of the column, we figured out 5 different possible values for the variable. Here we only fix it, not decide whether to keep all or remove a specific type (to be done later in the report).

In [17]:
vc_column(df, 'fuelType').head(10)

fuelType
Petrol    54108
Diesel    40877
Hybrid     2903
PETROL     1142
etrol      1124
Petro      1123
petrol     1087
iesel       850
DIESEL      837
diesel      829
Name: count, dtype: int64

In [18]:
canonical_fuel_types = ["Petrol", "Diesel", "Hybrid", "Eletric", "Other"]
fueltype_mapping = build_sim_mapping(df, 'fuelType', canonical_fuel_types)
fueltype_mapping.tail(10)

,Variation,AssignedValue
25,etro,Petrol
26,ETRO,Petrol
27,diese,Diesel
28,Othe,Other
29,YBRID,Hybrid
30,HYBRI,Hybrid
31,OTHER,Other
32,other,Other
33,IESE,Diesel
34,hybri,Hybrid


In [19]:
fueltype_mapping.to_csv('./mapping_dicts/fueltype_mapping.csv', index=False)

### 2.1.4. `transmission`

In [20]:
vc_column(df, 'transmission').head(20)

transmission
Manual       54362
Semi-Auto    22280
Automatic    19943
MANUAL        1143
manual        1142
anual         1117
Manua         1098
unknown       1011
 Manual        538
Semi-Aut       463
SEMI-AUTO      461
semi-auto      453
utomatic       445
emi-Auto       435
Automati       419
AUTOMATIC      412
automatic      396
unknow          27
UNKNOWN         22
anua            21
Name: count, dtype: int64

In [21]:
canonical_transmissions = ["Manual", "Automatic", "Semi-Auto", "Unknown"]

transmission_mapping = build_sim_mapping(df, 'transmission', canonical_transmissions)
transmission_mapping.tail(10)

,Variation,AssignedValue
30,nknown,Unknown
31,automati,Automatic
32,Other,Manual
33,semi-aut,Semi-Auto
34,Manual,Manual
35,AUTOMATI,Automatic
36,utomati,Automatic
37,Manual,Manual
38,manual,Manual
39,nknow,Unknown


In [22]:
transmission_mapping.to_csv('./mapping_dicts/transmission_mapping.csv', index=False)

### 2.1.5. `Model` Feature

This feature proved to be the hardest one. In order to properly cluster, we decided to create a dictionary that maps each brand to its possible models. This was done manually.

In [23]:
canonical_models_dict = {
    "Ford": ["focus", "fiesta", "kuga", "ecosport", "ka", "c-max", "mondeo", "b-max", "s-max", "grandcmax", "galaxy", "edge", "grandtourneoconnect", "puma", "tourneocustom", "mustang", "fusion", "streetka", "ranger", "escort"],

    "Mercedes": ["000", "c class", "a class", "e class", "glc class", "gla class", "b class", "cl class", "gle class", "sl class", "cls class", "v class", "s class", "gl class", "slk", "cla class", "x class", "gls class", "m class", "glb class", "g class", "clk", "sl", "clc class"],
    
    "VW": ["golf", "polo", "tiguan", "passat", "up", "troc", "touareg", "touran", "tcross", "sharan", "golfsv", "arteon", "scirocco", "amarok", "gol", "caravelle", "tiguanallspace", "cc", "beetle", "caddymaxilife", "shuttle", "jetta", "california", "caddylife", "eos", "caddy", "caddymaxi", "fox"],
    
    "Opel": ["corsa", "mokkax", "astra", "insignia", "crosslandx", "zafira", "grandland","grandlandx", "adam", "viva", "meriva", "combolife", "gtc", "zafiratourer", "agila", "antara", "vivaro", "cascada", "vectra", "tigra", "ampera"],
    
    "BMW": ["3 series", "1 series", "2 series", "5 series", "4 series", "x1", "x3", "x5", "x2", "x4", "m4", "6 series", "z4", "x6", "7 series", "x7", "8 series", "i3", "m3", "m5", "i8", "m2", "z3", "m6"],
    
    "Audi": ["a3", "q3", "a4", "a1", "q5", "a5", "q2", "a6", "q7", "tt", "a7", "a8", "q8", "rs6", "rs3", "r8", "rs4", "rs5", "s3", "s4", "sq5", "sq7", "s8", "a2", "s5"],
    
    "Toyota": ["yaris", "aygo", "auris", "chr", "rav4", "corolla", "prius", "verso", "avensis", "hilux", "gt86", "landcruiser", "proaceverso", "supra", "camry", "urbancruiser", "versos", "iq"],
    
    "Skoda": ["fabia", "octavia", "superb", "yetioutdoor", "citigo", "kodiaq", "karoq", "scala", "kamiq", "rapid", "yeti", "roomster"],
    
    "Hyundai": ["tucson", "i10", "i30", "i20", "kona", "ioniq", "santafe", "ix20", "i40", "ix35", "i800", "getz", "accent", "terracan"]
}

To ensure that the mapping is done correctly, we will iterate through each brand and build a mapping only for the models corresponding to that brand. That way the conversion is less prone to convert alike models into wrong brands (e.g. Audi Q1, BMW X1).

In [24]:
all_mappings = []  # list to collect all brand mapping DataFrames

for brand in canonical_brands:
    # get canonical models
    canonical_models = canonical_models_dict[brand]

    # get subset of df
    brand_df = df[df["Brand"] == brand]

    # model mapping using sim function
    model_mapping = build_sim_mapping(brand_df, 'model', canonical_models)

    # add brand info
    model_mapping["Brand"] = brand

    # collect
    all_mappings.append(model_mapping)

# combine all brands into one DataFrame
combined_mapping_df = pd.concat(all_mappings, ignore_index=True)

# reorder columns 
combined_mapping_df = combined_mapping_df[["Brand", "Variation", "AssignedValue"]]


**NOTES**

- Creation of a 000 model for mercedes, to make sure that all outlier number models (180, ...) are mapped to a incorrect class, instead of converting to a random c-class.
- The model mapping dataframe has a different structure than the rest (one more column).
- To handle these large amount of models, there are 2 alternatives:
  
   1. Create clusters of models (that have alike characteristics, might be from different brands). Then, drop models and keep that cluster ID.

   2. Target Encoding: Making sure to respect the different test, train and validation set rules, replace the model column with the median/mean/Xth Percentile of the price for that model.

In [25]:
combined_mapping_df.to_csv('./mapping_dicts/model_mapping.csv', index=False)